In [1]:
'''
Gurobi rendering of MiniZinc model

'''

import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [16]:
'''
wedding-seat-data.dzn
'''

guest_count = 10
child_count = 5
table_size = 5
child_table_size = 5

table_count = int(guest_count/table_size)

if child_table_size != 0:
    child_table_count = int(child_count/child_table_size)
else:
    child_table_count = 0

relationships_matrix = np.array(
[[ 0, -0.5, 0, 0.5, 0, 0, 0, 0, 0, 0],
 [-0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0],  
 [0, 0, 0, -0.5, 0, 0, 0, 0, 0, 0],  
 [ 0.5, 0, -0.5, 0, 0, 0, 0, 0, 0, 0],  
 [ 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0],  
 [ 0, 0, 0, 0, 0.25, 0, -0.5, 0, 0, 0],  
 [ 0, 0, 0, 0, 0, -0.5, 0, 0, 0, 0],  
 [ 0, 0, 0, 0, 0, 0, 0, 0, -0.5, 0],  
 [ 0, 0, 0, 0, 0, 0, 0, -0.5, 0, 0],  
 [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
)


In [17]:
'''
Parameter Sets
'''

GUESTS = [guest for guest in range(guest_count)]
CHILDREN = [child for child in range(guest_count, guest_count+child_count)]
TABLES = [table for table in range(table_count)]
CHILD_TABLES = [child_table for child_table in range(table_count, table_count+child_table_count)]
print(GUESTS)
print(CHILDREN)
print(TABLES)
print(CHILD_TABLES)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14]
[0, 1]
[2]


In [18]:
'''
Model
'''

m = gp.Model("wedding-seat")

In [19]:
'''
Decision Variables
'''

table_plan = m.addMVar((table_count+child_table_count, guest_count+child_count), vtype = GRB.BINARY)

same_table = m.addMVar((guest_count+child_count, guest_count+child_count, table_count+child_table_count), vtype = GRB.BINARY)

table_unhappiness = gp.LinExpr();

for t in TABLES:
    table_unhappiness += gp.quicksum([relationships_matrix[i,j] * same_table[i,j,t] for i in GUESTS for j in GUESTS]);

m.setObjective(table_unhappiness, GRB.MINIMIZE);

In [20]:
'''
Constraints
'''

##Each guest must be assigned to a table
for guest in GUESTS + CHILDREN:
    m.addConstr(gp.quicksum([table_plan[table,guest] for table in TABLES+CHILD_TABLES]) == 1)

##Each table must be at most table_size big
for table in TABLES:
    m.addConstr(gp.quicksum([table_plan[table,guest] for guest in GUESTS]) == table_size) 

##Each child table must be at most child_table_size big
for child_table in CHILD_TABLES:
    m.addConstr(gp.quicksum([table_plan[child_table,child] for child in CHILDREN]) == child_table_size)

##Adults and children sat apart (aka children sit exclusively with children)
for adult in GUESTS:
    for child in CHILDREN:
        for table in TABLES + CHILD_TABLES:
            m.addConstr(same_table[adult,child,table] == 0)
            
m.addConstr(table_plan[0,0] * table_plan[0,1] == 1)

##Links the table plan with the indicator variable array
for g1 in GUESTS:
    for g2 in GUESTS:
        for t in TABLES:
            m.addConstr(same_table[g1,g2,t] == table_plan[t, g1] * table_plan[t, g2])




In [21]:
'''
Run Model
'''

m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 168 rows, 720 columns and 220 nonzeros
Model fingerprint: 0xfe8906c5
Model has 201 quadratic constraints
Variable types: 0 continuous, 720 integer (720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 158 rows and 648 columns
Presolve time: 0.00s
Presolved: 234 rows, 128 columns, 536 nonzeros
Variable types: 0 continuous, 128 integer (128 binary)
Found heuristic solution: objective 0.0000000
Found heuristic solution: objective -3.0000000

Root relaxation: objective -4.000000e+00, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

In [22]:
print(table_plan.X)

[[1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]]
